In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn import svm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
%matplotlib inline 
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import KMeansSMOTE
from imblearn.over_sampling import SMOTENC
from collections import Counter
from smote_variants.visualization import (plot_raw, 
                                          plot_oversampled, 
                                          plot_sampling,
                                          plot_sampling_raw, 
                                          plot_comparison, 
                                          plot_comparison_raw)
import smote_variants as sv
from imblearn.over_sampling import SMOTE
from collections import Counter

In [4]:
#data = pd.ExcelFile()
data = pd.read_excel('Earthquake Damage Data.xlsx', 'Data')
data.head()

,ID,P1,P2,P3,P4,P5,P6,BC1,BC2,BC3,BC4,BC5,BC6,BC7,BC8,DS
0,1,0.733800,0.588313,0.824660,0.420993,0.00,0.599542,0.257812,0,0.0,0,0.8,1.0,0.000000,0.827586,4
1,2,0.738113,0.604497,0.477239,0.252442,0.75,0.544622,0.148438,0,0.0,1,0.8,1.0,0.666667,0.931034,1
2,3,0.742815,0.602485,0.755395,0.434462,0.75,0.560641,0.093750,1,1.0,1,0.8,1.0,0.000000,0.655172,1
3,4,0.739635,0.602389,0.768629,0.484940,0.75,0.556064,0.148438,0,0.0,0,0.8,1.0,0.000000,0.965517,1
4,5,0.740052,0.605121,0.783227,0.486592,0.75,0.542334,0.148438,0,0.0,1,0.8,0.5,0.666667,0.931034,1


In [5]:
encode_array = OneHotEncoder()
x = data.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,12,13,14]].values
y = data.iloc[:,15].values

print (x.shape)
print (y.shape)

(30590, 14)
(30590,)


In [6]:

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)
# 获取维度参数
num_features = x_train.shape[1]  # 特征数量（14）
num_classes = len(np.unique(y))  # 类别数量（3）


In [7]:
smote = SMOTE(
    sampling_strategy={2: 4000, 3: 5000, 4: 15000},
    random_state=42
)
x_resampled, y_resampled = smote.fit_resample(x_train, y_train)
print("过采样后分布:", Counter(y_resampled))

过采样后分布: Counter({1: 22156, 4: 15000, 3: 5000, 2: 4000})


In [8]:
oversampler_0= sv.MulticlassOversampling('SMOTE')
x_train_resampled_0, y_train_resampled_0 = oversampler_0.sample(x_train, y_train)
oversampler_1= sv.MulticlassOversampling('distance_SMOTE')
x_train_resampled_1, y_train_resampled_1 = oversampler_1.sample(x_train, y_train)   

2025-04-20 16:18:50,853:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2025-04-20 16:18:50,855:INFO:MulticlassOversampling: Sampling minority class with label: 4
2025-04-20 16:18:50,857:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2025-04-20 16:18:50,858:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2025-04-20 16:18:50,860:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2025-04-20 16:18:50,886:INFO:SMOTE: simplex sampling with n_dim 2
2025-04-20 16:18:50,921:INFO:MulticlassOversampling: Sampling minority class with label: 3
2025-04-20 16:18:50,923:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 0.498347144862564, 'n_neighbors': 5, 'nn_params': {}, 'n_

## KNN

In [9]:
model_KNN = KNeighborsClassifier(n_neighbors = 2)
model_KNN.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=2)

In [10]:
y_pred_KNN = model_KNN.predict(x_test)

In [11]:
print("\nClassification Report:\n", classification_report(y_test, y_pred_KNN))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_KNN))


Classification Report:
               precision    recall  f1-score   support

           1       0.91      0.98      0.95      5539
           2       0.27      0.12      0.17        24
           3       0.00      0.00      0.00        37
           4       0.35      0.09      0.14       518

    accuracy                           0.90      6118
   macro avg       0.38      0.30      0.31      6118
weighted avg       0.86      0.90      0.87      6118


Confusion Matrix:
 [[5438    8    7   86]
 [  21    3    0    0]
 [  36    0    0    1]
 [ 470    0    2   46]]


In [12]:
model_KNN.fit(x_resampled, y_resampled)
pred_0 = model_KNN.predict(x_test)

# 打印分类报告
print(metrics.classification_report(y_test, pred_0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_0))

              precision    recall  f1-score   support

           1       0.92      0.88      0.90      5539
           2       0.08      0.21      0.11        24
           3       0.02      0.05      0.03        37
           4       0.22      0.28      0.24       518

    accuracy                           0.82      6118
   macro avg       0.31      0.36      0.32      6118
weighted avg       0.85      0.82      0.84      6118


Confusion Matrix:
 [[4890   58   79  512]
 [  17    5    2    0]
 [  33    0    2    2]
 [ 365    1    9  143]]


In [13]:
model_KNN.fit(x_train_resampled_0, y_train_resampled_0)
pred_1 = model_KNN.predict(x_test)  
        # 打印分类报告
print(metrics.classification_report(y_test, pred_1))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_1))

              precision    recall  f1-score   support

           1       0.92      0.87      0.90      5539
           2       0.06      0.17      0.09        24
           3       0.02      0.05      0.03        37
           4       0.22      0.31      0.26       518

    accuracy                           0.81      6118
   macro avg       0.31      0.35      0.32      6118
weighted avg       0.86      0.81      0.83      6118


Confusion Matrix:
 [[4808   65  106  560]
 [  16    4    3    1]
 [  32    0    2    3]
 [ 345    1   11  161]]


In [14]:
model_KNN.fit(x_train_resampled_1, y_train_resampled_1)
pred_2 = model_KNN.predict(x_test)  
# 打印分类报告
print(metrics.classification_report(y_test, pred_2))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_2))

              precision    recall  f1-score   support

           1       0.92      0.89      0.91      5539
           2       0.08      0.17      0.11        24
           3       0.03      0.05      0.04        37
           4       0.23      0.28      0.25       518

    accuracy                           0.83      6118
   macro avg       0.31      0.35      0.33      6118
weighted avg       0.85      0.83      0.84      6118


Confusion Matrix:
 [[4944   46   57  492]
 [  18    4    2    0]
 [  31    0    2    4]
 [ 367    1    5  145]]


# SVM

In [15]:
model_SVM = svm.SVC(kernel='linear')
model_SVM.fit(x_train, y_train)

SVC(kernel='linear')

In [16]:
y_pred_SVM = model_SVM.predict(x_test)

print("\nClassification Report:\n", classification_report(y_test, y_pred_SVM))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_SVM))


Classification Report:
               precision    recall  f1-score   support

           1       0.91      1.00      0.95      5539
           2       0.00      0.00      0.00        24
           3       0.00      0.00      0.00        37
           4       0.00      0.00      0.00       518

    accuracy                           0.91      6118
   macro avg       0.23      0.25      0.24      6118
weighted avg       0.82      0.91      0.86      6118


Confusion Matrix:
 [[5539    0    0    0]
 [  24    0    0    0]
 [  37    0    0    0]
 [ 518    0    0    0]]


d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
model_SVM.fit(x_resampled, y_resampled)
pred_0 = model_SVM.predict(x_test)

# 打印分类报告
print(metrics.classification_report(y_test, pred_0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_0))

              precision    recall  f1-score   support

           1       0.92      0.75      0.83      5539
           2       0.04      0.71      0.08        24
           3       0.00      0.00      0.00        37
           4       0.16      0.37      0.22       518

    accuracy                           0.72      6118
   macro avg       0.28      0.46      0.28      6118
weighted avg       0.85      0.72      0.77      6118


Confusion Matrix:
 [[4178  357    0 1004]
 [   7   17    0    0]
 [  27    1    0    9]
 [ 316   11    0  191]]


d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
model_SVM.fit(x_train_resampled_0, y_train_resampled_0)
pred_1 = model_SVM.predict(x_test)  
        # 打印分类报告
print(metrics.classification_report(y_test, pred_1))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_1))

              precision    recall  f1-score   support

           1       0.93      0.39      0.55      5539
           2       0.02      0.88      0.05        24
           3       0.01      0.46      0.02        37
           4       0.15      0.42      0.22       518

    accuracy                           0.40      6118
   macro avg       0.28      0.54      0.21      6118
weighted avg       0.85      0.40      0.52      6118


Confusion Matrix:
 [[2177  844 1271 1247]
 [   3   21    0    0]
 [  10    1   17    9]
 [ 151   24  125  218]]


In [19]:
model_SVM.fit(x_train_resampled_1, y_train_resampled_1)
pred_2 = model_SVM.predict(x_test)  
# 打印分类报告
print(metrics.classification_report(y_test, pred_2))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_2))

              precision    recall  f1-score   support

           1       0.93      0.39      0.55      5539
           2       0.02      0.83      0.04        24
           3       0.01      0.43      0.02        37
           4       0.15      0.41      0.22       518

    accuracy                           0.39      6118
   macro avg       0.28      0.52      0.21      6118
weighted avg       0.85      0.39      0.52      6118


Confusion Matrix:
 [[2157  835 1312 1235]
 [   4   20    0    0]
 [  10    1   16   10]
 [ 153   24  127  214]]


# linear regression

In [20]:
model_lr = LinearRegression()
model_lr.fit(x_train, y_train)
y_pred_lr = model_lr.predict(x_test)
print("预测值示例:", y_pred_lr[:5])


预测值示例: [1.05470751 1.17007976 1.49089912 1.21049263 1.33937797]


#### 预测值是分数（连续数值），而不是离散的类别标签 ​线性回归模型（LinearRegression）​，而线性回归是专门用于解决 ​回归任务 的模型（预测连续值）

# logistic regression

In [21]:
from sklearn.linear_model import LogisticRegression
model_lg = LogisticRegression(multi_class='multinomial', max_iter=1000)
model_lg.fit(x_train, y_train)
y_pred_lg = model_lg.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred_lg))
print("\nClassification Report:\n", classification_report(y_test, y_pred_lg))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_lg))

d:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.9053612291598562

Classification Report:
               precision    recall  f1-score   support

           1       0.91      1.00      0.95      5539
           2       0.00      0.00      0.00        24
           3       0.00      0.00      0.00        37
           4       0.00      0.00      0.00       518

    accuracy                           0.91      6118
   macro avg       0.23      0.25      0.24      6118
weighted avg       0.82      0.91      0.86      6118


Confusion Matrix:
 [[5539    0    0    0]
 [  24    0    0    0]
 [  37    0    0    0]
 [ 518    0    0    0]]


d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
model_lg.fit(x_resampled, y_resampled)
pred_0 = model_lg.predict(x_test)

# 打印分类报告
print(metrics.classification_report(y_test, pred_0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_0))

d:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


              precision    recall  f1-score   support

           1       0.92      0.75      0.83      5539
           2       0.02      0.38      0.05        24
           3       0.00      0.00      0.00        37
           4       0.15      0.36      0.22       518

    accuracy                           0.71      6118
   macro avg       0.27      0.37      0.27      6118
weighted avg       0.85      0.71      0.77      6118


Confusion Matrix:
 [[4145  352    8 1034]
 [  15    9    0    0]
 [  26    0    0   11]
 [ 319   10    0  189]]


In [23]:
model_lg.fit(x_train_resampled_0, y_train_resampled_0)
pred_1 = model_lg.predict(x_test)  
        # 打印分类报告
print(metrics.classification_report(y_test, pred_1))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_1))

d:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


              precision    recall  f1-score   support

           1       0.94      0.32      0.48      5539
           2       0.02      0.92      0.04        24
           3       0.01      0.41      0.02        37
           4       0.14      0.45      0.21       518

    accuracy                           0.34      6118
   macro avg       0.28      0.52      0.19      6118
weighted avg       0.86      0.34      0.45      6118


Confusion Matrix:
 [[1791  932 1366 1450]
 [   1   22    0    1]
 [   7    1   15   14]
 [ 106   27  152  233]]


In [24]:
model_lg.fit(x_train_resampled_1, y_train_resampled_1)
pred_2 = model_lg.predict(x_test)  
# 打印分类报告
print(metrics.classification_report(y_test, pred_2))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_2))

d:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


              precision    recall  f1-score   support

           1       0.94      0.32      0.48      5539
           2       0.02      0.92      0.04        24
           3       0.01      0.35      0.02        37
           4       0.13      0.44      0.21       518

    accuracy                           0.34      6118
   macro avg       0.28      0.51      0.19      6118
weighted avg       0.86      0.34      0.45      6118


Confusion Matrix:
 [[1785  921 1365 1468]
 [   1   22    0    1]
 [   9    1   13   14]
 [ 109   27  152  230]]


# Naive bayes

In [25]:
from sklearn.naive_bayes import GaussianNB
model_nv = GaussianNB()
model_nv.fit(x_train, y_train)
y_pred_nv = model_nv.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred_nv))
print("\nClassification Report:\n", classification_report(y_test, y_pred_nv))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_nv))

Accuracy: 0.4818568159529258

Classification Report:
               precision    recall  f1-score   support

           1       0.91      0.50      0.65      5539
           2       0.01      0.96      0.02        24
           3       0.00      0.00      0.00        37
           4       0.14      0.27      0.19       518

    accuracy                           0.48      6118
   macro avg       0.27      0.43      0.21      6118
weighted avg       0.84      0.48      0.60      6118


Confusion Matrix:
 [[2784 1922    5  828]
 [   1   23    0    0]
 [  18    5    0   14]
 [ 252  125    0  141]]


In [26]:
model_nv.fit(x_resampled, y_resampled)
pred_0 = model_nv.predict(x_test)

# 打印分类报告
print(metrics.classification_report(y_test, pred_0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_0))

              precision    recall  f1-score   support

           1       0.93      0.27      0.42      5539
           2       0.01      0.92      0.02        24
           3       0.03      0.16      0.05        37
           4       0.12      0.52      0.19       518

    accuracy                           0.30      6118
   macro avg       0.27      0.47      0.17      6118
weighted avg       0.85      0.30      0.40      6118


Confusion Matrix:
 [[1523 1861  181 1974]
 [   2   22    0    0]
 [   6    4    6   21]
 [ 107  127   13  271]]


In [27]:
model_nv.fit(x_train_resampled_0, y_train_resampled_0)
pred_1 = model_nv.predict(x_test)  
        # 打印分类报告
print(metrics.classification_report(y_test, pred_1))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_1))

              precision    recall  f1-score   support

           1       0.94      0.22      0.35      5539
           2       0.01      0.96      0.02        24
           3       0.01      0.27      0.03        37
           4       0.12      0.50      0.20       518

    accuracy                           0.24      6118
   macro avg       0.27      0.49      0.15      6118
weighted avg       0.86      0.24      0.34      6118


Confusion Matrix:
 [[1198 1881  632 1828]
 [   1   23    0    0]
 [   3    4   10   20]
 [  78  127   55  258]]


In [28]:
model_nv.fit(x_train_resampled_1, y_train_resampled_1)
pred_2 = model_nv.predict(x_test)  
# 打印分类报告
print(metrics.classification_report(y_test, pred_2))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_2))

              precision    recall  f1-score   support

           1       0.94      0.23      0.37      5539
           2       0.01      0.92      0.02        24
           3       0.01      0.27      0.03        37
           4       0.13      0.49      0.20       518

    accuracy                           0.26      6118
   macro avg       0.27      0.48      0.15      6118
weighted avg       0.86      0.26      0.35      6118


Confusion Matrix:
 [[1284 1845  668 1742]
 [   2   22    0    0]
 [   4    4   10   19]
 [  78  126   62  252]]


# Decision tree

In [29]:
from sklearn.tree import DecisionTreeClassifier
model_DT = DecisionTreeClassifier(
    random_state=42,
    class_weight='balanced'  # 自动平衡类别权重
)
model_DT.fit(x_train, y_train)
y_pred_DT = model_DT.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred_DT))
print("\nClassification Report:\n", classification_report(y_test, y_pred_DT))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_DT))

Accuracy: 0.8592677345537757

Classification Report:
               precision    recall  f1-score   support

           1       0.92      0.92      0.92      5539
           2       0.05      0.04      0.04        24
           3       0.03      0.03      0.03        37
           4       0.26      0.27      0.27       518

    accuracy                           0.86      6118
   macro avg       0.32      0.31      0.32      6118
weighted avg       0.86      0.86      0.86      6118


Confusion Matrix:
 [[5117   20   26  376]
 [  18    1    0    5]
 [  33    0    1    3]
 [ 377    0    3  138]]


In [30]:
model_DT.fit(x_resampled, y_resampled)
pred_0 = model_DT.predict(x_test)

# 打印分类报告
print(metrics.classification_report(y_test, pred_0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_0))

              precision    recall  f1-score   support

           1       0.93      0.85      0.89      5539
           2       0.05      0.12      0.07        24
           3       0.04      0.11      0.06        37
           4       0.23      0.37      0.28       518

    accuracy                           0.81      6118
   macro avg       0.31      0.37      0.32      6118
weighted avg       0.86      0.81      0.83      6118


Confusion Matrix:
 [[4729   59   92  659]
 [  18    3    0    3]
 [  30    0    4    3]
 [ 312    0   12  194]]


In [31]:
model_DT.fit(x_train_resampled_0, y_train_resampled_0)
pred_1 = model_DT.predict(x_test)  
        # 打印分类报告
print(metrics.classification_report(y_test, pred_1))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_1))

              precision    recall  f1-score   support

           1       0.93      0.85      0.89      5539
           2       0.11      0.25      0.15        24
           3       0.02      0.05      0.03        37
           4       0.21      0.37      0.27       518

    accuracy                           0.80      6118
   macro avg       0.32      0.38      0.33      6118
weighted avg       0.86      0.80      0.83      6118


Confusion Matrix:
 [[4684   50  105  700]
 [  13    6    1    4]
 [  29    0    2    6]
 [ 314    0   10  194]]


In [32]:
model_DT.fit(x_train_resampled_1, y_train_resampled_1)
pred_2 = model_DT.predict(x_test)  
# 打印分类报告
print(metrics.classification_report(y_test, pred_2))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_2))

              precision    recall  f1-score   support

           1       0.93      0.90      0.91      5539
           2       0.12      0.12      0.12        24
           3       0.04      0.05      0.05        37
           4       0.25      0.31      0.28       518

    accuracy                           0.84      6118
   macro avg       0.33      0.35      0.34      6118
weighted avg       0.86      0.84      0.85      6118


Confusion Matrix:
 [[4987   22   36  494]
 [  18    3    0    3]
 [  30    0    2    5]
 [ 347    0    8  163]]


# ANN(MLP)

In [33]:
from sklearn.neural_network import MLPClassifier
model_MLP = MLPClassifier(
    hidden_layer_sizes=(50, 20),  # 隐藏层结构
    activation='relu',
    solver='adam',
    max_iter=200,
    random_state=42,
    
   
)

model_MLP.fit(x_train, y_train)
y_pred_MLP = model_MLP.predict(x_test)
print("\nClassification Report:\n", classification_report(y_test, y_pred_MLP))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_MLP))


Classification Report:
               precision    recall  f1-score   support

           1       0.91      1.00      0.95      5539
           2       0.00      0.00      0.00        24
           3       0.00      0.00      0.00        37
           4       0.00      0.00      0.00       518

    accuracy                           0.90      6118
   macro avg       0.23      0.25      0.24      6118
weighted avg       0.82      0.90      0.86      6118


Confusion Matrix:
 [[5531    0    0    8]
 [  24    0    0    0]
 [  37    0    0    0]
 [ 518    0    0    0]]


d:\anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

In [34]:
model_MLP.fit(x_resampled, y_resampled)
pred_0 = model_MLP.predict(x_test)

# 打印分类报告
print(metrics.classification_report(y_test, pred_0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_0))

              precision    recall  f1-score   support

           1       0.94      0.71      0.81      5539
           2       0.07      0.67      0.12        24
           3       0.02      0.16      0.04        37
           4       0.19      0.51      0.28       518

    accuracy                           0.69      6118
   macro avg       0.31      0.51      0.31      6118
weighted avg       0.87      0.69      0.76      6118


Confusion Matrix:
 [[3959  213  251 1116]
 [   7   16    1    0]
 [  21    0    6   10]
 [ 233    3   16  266]]


d:\anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [35]:
model_MLP.fit(x_train_resampled_0, y_train_resampled_0)
pred_1 = model_MLP.predict(x_test)  
        # 打印分类报告
print(metrics.classification_report(y_test, pred_1))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_1))

              precision    recall  f1-score   support

           1       0.95      0.58      0.72      5539
           2       0.04      0.62      0.08        24
           3       0.02      0.38      0.04        37
           4       0.18      0.61      0.28       518

    accuracy                           0.58      6118
   macro avg       0.30      0.55      0.28      6118
weighted avg       0.88      0.58      0.68      6118


Confusion Matrix:
 [[3213  334  541 1451]
 [   6   15    2    1]
 [  14    0   14    9]
 [ 150    3   49  316]]


d:\anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [36]:
model_MLP.fit(x_train_resampled_1, y_train_resampled_1)
pred_2 = model_MLP.predict(x_test)  
# 打印分类报告
print(metrics.classification_report(y_test, pred_2))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_2))

              precision    recall  f1-score   support

           1       0.95      0.61      0.74      5539
           2       0.05      0.54      0.08        24
           3       0.02      0.24      0.03        37
           4       0.18      0.59      0.28       518

    accuracy                           0.61      6118
   macro avg       0.30      0.50      0.28      6118
weighted avg       0.87      0.61      0.70      6118


Confusion Matrix:
 [[3388  265  500 1386]
 [   7   13    3    1]
 [  18    0    9   10]
 [ 157    5   49  307]]


d:\anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


# CNN

In [37]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# 构建 1D-CNN 模型
model_CNN = models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation='relu', 
                  kernel_regularizer=regularizers.l2(0.001),
                  input_shape=(14, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.3),
    layers.Conv1D(64, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model_CNN.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 训练
history = model_CNN.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)
model_CNN.summary()
# 预测与评估
y_pred_CNN = model_CNN.predict(x_test)
y_pred_CNN = np.argmax(y_pred_CNN, axis=1)

print("Accuracy:", accuracy_score(y_test, y_pred_CNN))
print("\nClassification Report:\n", classification_report(y_test, y_pred_CNN))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_CNN))

Epoch 1/10


d:\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


612/612 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8565 - loss: 0.5832 - val_accuracy: 0.9015 - val_loss: 0.3611
Epoch 2/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9060 - loss: 0.3629 - val_accuracy: 0.9015 - val_loss: 0.3496
Epoch 3/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9039 - loss: 0.3649 - val_accuracy: 0.9015 - val_loss: 0.3403
Epoch 4/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9067 - loss: 0.3505 - val_accuracy: 0.9015 - val_loss: 0.3395
Epoch 5/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9071 - loss: 0.3436 - val_accuracy: 0.9015 - val_loss: 0.3337
Epoch 6/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9058 - loss: 0.3450 - val_accuracy: 0.9015 - val_loss: 0.3422
Epoch 7/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9065 - loss: 0.3386 - val_accuracy: 0.9015 - val_loss: 0.3328
Epoch 8/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9025 - loss: 0.3514 - val_accuracy: 0.9015 - val_

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 12, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 6, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 4, 64)          │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 4, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 2, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Trainable params: 15,434 (60.29 KB)

 Non-trainable params: 192 (768.00 B)

 Optimizer params: 30,870 (120.59 KB)

192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9053612291598562

Classification Report:
               precision    recall  f1-score   support

           1       0.91      1.00      0.95      5539
           2       0.00      0.00      0.00        24
           3       0.00      0.00      0.00        37
           4       0.00      0.00      0.00       518

    accuracy                           0.91      6118
   macro avg       0.23      0.25      0.24      6118
weighted avg       0.82      0.91      0.86      6118


Confusion Matrix:
 [[5539    0    0    0]
 [  24    0    0    0]
 [  37    0    0    0]
 [ 518    0    0    0]]


d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [38]:


model_CNN.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 训练
history = model_CNN.fit(
    x_resampled, y_resampled,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)
y_pred_CNN = model_CNN.predict(x_test)
y_pred_CNN = np.argmax(y_pred_CNN, axis=1)


print("\nClassification Report:\n", classification_report(y_test, y_pred_CNN))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_CNN))

Epoch 1/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6100 - loss: 0.9712 - val_accuracy: 0.1179 - val_loss: 1.5202
Epoch 2/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6291 - loss: 0.8666 - val_accuracy: 0.0261 - val_loss: 1.5649
Epoch 3/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6394 - loss: 0.8346 - val_accuracy: 0.1121 - val_loss: 1.5498
Epoch 4/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6442 - loss: 0.8140 - val_accuracy: 0.1229 - val_loss: 1.5230
Epoch 5/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6487 - loss: 0.7953 - val_accuracy: 0.1817 - val_loss: 1.3797
Epoch 6/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6507 - loss: 0.7873 - val_accuracy: 0.1663 - val_loss: 1.4124
Epoch 7/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6571 - loss: 0.7778 - val_accuracy: 0.1873 - val_loss: 1.3679
Epoch 8/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6614 - loss: 0.7660 - 

In [39]:
model_CNN.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 训练
history = model_CNN.fit(
    x_train_resampled_0, y_train_resampled_0,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)
y_pred_CNN = model_CNN.predict(x_test)
y_pred_CNN = np.argmax(y_pred_CNN, axis=1)

print("Accuracy:", accuracy_score(y_test, y_pred_CNN))
print("\nClassification Report:\n", classification_report(y_test, y_pred_CNN))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_CNN))

Epoch 1/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.5834 - loss: 0.8814 - val_accuracy: 0.9048 - val_loss: 0.4806
Epoch 2/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6150 - loss: 0.8428 - val_accuracy: 0.9337 - val_loss: 0.4756
Epoch 3/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.6223 - loss: 0.8304 - val_accuracy: 0.9167 - val_loss: 0.4993
Epoch 4/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6257 - loss: 0.8220 - val_accuracy: 0.9639 - val_loss: 0.4734
Epoch 5/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6286 - loss: 0.8150 - val_accuracy: 0.9105 - val_loss: 0.4866
Epoch 6/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6383 - loss: 0.7974 - val_accuracy: 0.9252 - val_loss: 0.4930
Epoch 7/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6390 - loss: 0.7979 - val_accuracy: 0.9211 - val_loss: 0.4956
Epoch 8/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6411 - loss: 0.7942 - 

In [40]:
model_CNN.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 训练
history = model_CNN.fit(
    x_train_resampled_1, y_train_resampled_1,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)
y_pred_CNN = model_CNN.predict(x_test)
y_pred_CNN = np.argmax(y_pred_CNN, axis=1)

print("Accuracy:", accuracy_score(y_test, y_pred_CNN))
print("\nClassification Report:\n", classification_report(y_test, y_pred_CNN))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_CNN))

Epoch 1/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.6690 - loss: 0.7422 - val_accuracy: 0.9457 - val_loss: 0.4792
Epoch 2/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6748 - loss: 0.7309 - val_accuracy: 0.9485 - val_loss: 0.4753
Epoch 3/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6792 - loss: 0.7234 - val_accuracy: 0.9325 - val_loss: 0.4384
Epoch 4/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6824 - loss: 0.7203 - val_accuracy: 0.9330 - val_loss: 0.4823
Epoch 5/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6820 - loss: 0.7166 - val_accuracy: 0.9811 - val_loss: 0.3439
Epoch 6/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.6913 - loss: 0.7033 - val_accuracy: 0.9146 - val_loss: 0.4659
Epoch 7/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.6864 - loss: 0.7126 - val_accuracy: 0.9520 - val_loss: 0.3621
Epoch 8/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.6926 - loss: 0.7000 

# XGBOOST

In [41]:

from xgboost import XGBClassifier
model_XG = XGBClassifier()
model_XG.fit(x_train.squeeze(), y_train-1)
y_pred_XG = model_XG.predict(x_test.squeeze())


print("Accuracy:", accuracy_score(y_test-1, y_pred_XG))
print("\nClassification Report:\n", classification_report(y_test-1, y_pred_XG))
print("\nConfusion Matrix:\n", confusion_matrix(y_test-1, y_pred_XG))

Accuracy: 0.905197777051324

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.99      0.95      5539
           1       0.00      0.00      0.00        24
           2       0.50      0.03      0.05        37
           3       0.50      0.11      0.18       518

    accuracy                           0.91      6118
   macro avg       0.48      0.28      0.29      6118
weighted avg       0.87      0.91      0.88      6118


Confusion Matrix:
 [[5481    1    1   56]
 [  24    0    0    0]
 [  36    0    1    0]
 [ 462    0    0   56]]


In [42]:
model_XG.fit(x_resampled, y_resampled-1)
y_pred_XG = model_XG.predict(x_test.squeeze())

# 打印分类报告
print(metrics.classification_report(y_test-1, y_pred_XG))
print("\nConfusion Matrix:\n", confusion_matrix(y_test-1, y_pred_XG))

              precision    recall  f1-score   support

           0       0.93      0.90      0.92      5539
           1       0.09      0.21      0.12        24
           2       0.05      0.03      0.03        37
           3       0.28      0.35      0.31       518

    accuracy                           0.85      6118
   macro avg       0.33      0.37      0.35      6118
weighted avg       0.86      0.85      0.86      6118


Confusion Matrix:
 [[5008   51   19  461]
 [  19    5    0    0]
 [  33    0    1    3]
 [ 336    1    2  179]]


In [43]:
model_XG.fit(x_train_resampled_0, y_train_resampled_0-1)
y_pred_XG = model_XG.predict(x_test.squeeze())

# 打印分类报告
print(metrics.classification_report(y_test-1, y_pred_XG))
print("\nConfusion Matrix:\n", confusion_matrix(y_test-1, y_pred_XG))

              precision    recall  f1-score   support

           0       0.93      0.86      0.90      5539
           1       0.06      0.25      0.10        24
           2       0.01      0.03      0.01        37
           3       0.27      0.42      0.33       518

    accuracy                           0.82      6118
   macro avg       0.32      0.39      0.34      6118
weighted avg       0.87      0.82      0.84      6118


Confusion Matrix:
 [[4783   88   88  580]
 [  16    6    2    0]
 [  32    0    1    4]
 [ 290    1    8  219]]


In [44]:
model_XG.fit(x_train_resampled_1, y_train_resampled_1-1)
y_pred_XG = model_XG.predict(x_test.squeeze())

# 打印分类报告
print(metrics.classification_report(y_test-1, y_pred_XG))
print("\nConfusion Matrix:\n", confusion_matrix(y_test-1, y_pred_XG))

              precision    recall  f1-score   support

           0       0.92      0.95      0.94      5539
           1       0.05      0.04      0.04        24
           2       0.00      0.00      0.00        37
           3       0.33      0.25      0.29       518

    accuracy                           0.88      6118
   macro avg       0.33      0.31      0.32      6118
weighted avg       0.86      0.88      0.87      6118


Confusion Matrix:
 [[5258   19    2  260]
 [  23    1    0    0]
 [  35    0    0    2]
 [ 386    1    0  131]]
